In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm

In [2]:
all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [3]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [4]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'

In [5]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [6]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [7]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6816 entries, 8 to 3035139
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype    
---  ------               --------------  -----    
 0   date                 6816 non-null   period[D]
 1   store_nbr            6816 non-null   category 
 2   family               6816 non-null   category 
 3   sales                6752 non-null   float64  
 4   onpromotion          6816 non-null   int64    
 5   sales_lag_01         6756 non-null   float64  
 6   sales_lag_02         6760 non-null   float64  
 7   sales_lag_03         6764 non-null   float64  
 8   sales_lag_04         6768 non-null   float64  
 9   sales_lag_05         6772 non-null   float64  
 10  sales_lag_06         6776 non-null   float64  
 11  sales_lag_07         6780 non-null   float64  
 12  sales_lag_08         6784 non-null   float64  
 13  sales_lag_09         6788 non-null   float64  
 14  sales_lag_10         6792 non-null   float64  
 15  s

## One Hot Encoding

In [8]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday'])    

In [9]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

## Experiment I -- Linear Regression

In [10]:
run_experiment_I = True
if run_experiment_I:
    lr = LinearRegression()
    lr.fit(X_train, y_train)

In [11]:
if run_experiment_I:
    y_pred_train = lr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.20928922205788442
RMS log-error val:  0.231758418939929
RMS log-error train (actual):  0.9087185771821745
RMS log-error val (actual):  0.9985451307445373


## Experiment II -- Lightgbm

In [12]:
lgb_params = {
    'boosting_type' : 'gbdt',  # gradient boosting decision tree
    'early_stopping_rounds': 200,
    'force_col_wise': True,
    'learning_rate': 0.1,
    'max_depth': 10,
    'metric': 'mse',  # mean square error
    'num_iterations': 5000,
    'num_leaves': 10,
    'random_state': 1,
    'verbose': 0
}

run_experiment_II = True  # should be true
if run_experiment_II:
    X_train_lgb = lightgbm.Dataset(data=X_train, label=y_train, feature_name='auto')
    X_val_lgb = lightgbm.Dataset(data=X_val, label=y_val, reference=X_train_lgb, feature_name='auto')

In [13]:
if run_experiment_II:
    lgb = lightgbm.train(
        params=lgb_params, 
        train_set=X_train_lgb,
        valid_sets=[X_train_lgb, X_val_lgb],
    )

[1]	training's l2: 3.08025	valid_1's l2: 3.32536
Training until validation scores don't improve for 200 rounds
[2]	training's l2: 2.64088	valid_1's l2: 2.8625
[3]	training's l2: 2.28494	valid_1's l2: 2.49129
[4]	training's l2: 1.99155	valid_1's l2: 2.18464
[5]	training's l2: 1.75269	valid_1's l2: 1.93936
[6]	training's l2: 1.55293	valid_1's l2: 1.73581
[7]	training's l2: 1.38871	valid_1's l2: 1.56638
[8]	training's l2: 1.2528	valid_1's l2: 1.43213
[9]	training's l2: 1.14397	valid_1's l2: 1.32266
[10]	training's l2: 1.05567	valid_1's l2: 1.23178
[11]	training's l2: 0.982369	valid_1's l2: 1.16004
[12]	training's l2: 0.919926	valid_1's l2: 1.10163
[13]	training's l2: 0.869092	valid_1's l2: 1.05099
[14]	training's l2: 0.826757	valid_1's l2: 1.01009
[15]	training's l2: 0.788786	valid_1's l2: 0.980336
[16]	training's l2: 0.753583	valid_1's l2: 0.945958
[17]	training's l2: 0.724031	valid_1's l2: 0.917791
[18]	training's l2: 0.69783	valid_1's l2: 0.900712
[19]	training's l2: 0.674961	valid_1's

/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[171]	training's l2: 0.199387	valid_1's l2: 0.764011
[172]	training's l2: 0.197458	valid_1's l2: 0.763854
[173]	training's l2: 0.1966	valid_1's l2: 0.761488
[174]	training's l2: 0.195455	valid_1's l2: 0.761336
[175]	training's l2: 0.194324	valid_1's l2: 0.763025
[176]	training's l2: 0.193571	valid_1's l2: 0.763779
[177]	training's l2: 0.192727	valid_1's l2: 0.763965
[178]	training's l2: 0.191959	valid_1's l2: 0.764188
[179]	training's l2: 0.191085	valid_1's l2: 0.76299
[180]	training's l2: 0.190247	valid_1's l2: 0.761166
[181]	training's l2: 0.189809	valid_1's l2: 0.761296
[182]	training's l2: 0.188997	valid_1's l2: 0.761071
[183]	training's l2: 0.18798	valid_1's l2: 0.760581
[184]	training's l2: 0.186727	valid_1's l2: 0.760571
[185]	training's l2: 0.185686	valid_1's l2: 0.760414
[186]	training's l2: 0.185	valid_1's l2: 0.760588
[187]	training's l2: 0.183842	valid_1's l2: 0.76059
[188]	training's l2: 0.182729	valid_1's l2: 0.760463
[189]	training's l2: 0.181973	valid_1's l2: 0.761721
[

[374]	training's l2: 0.0847432	valid_1's l2: 0.771207
[375]	training's l2: 0.0843536	valid_1's l2: 0.77109
[376]	training's l2: 0.0840959	valid_1's l2: 0.770744
[377]	training's l2: 0.0837883	valid_1's l2: 0.771273
[378]	training's l2: 0.0834566	valid_1's l2: 0.771106
[379]	training's l2: 0.0833779	valid_1's l2: 0.771085
[380]	training's l2: 0.0832138	valid_1's l2: 0.770778
[381]	training's l2: 0.082857	valid_1's l2: 0.770971
[382]	training's l2: 0.0827012	valid_1's l2: 0.771202
[383]	training's l2: 0.0824927	valid_1's l2: 0.771702
[384]	training's l2: 0.0821722	valid_1's l2: 0.771245
[385]	training's l2: 0.0817519	valid_1's l2: 0.770176
[386]	training's l2: 0.0814709	valid_1's l2: 0.770151
[387]	training's l2: 0.0810742	valid_1's l2: 0.769944
[388]	training's l2: 0.0809308	valid_1's l2: 0.769869
[389]	training's l2: 0.0807303	valid_1's l2: 0.769868
[390]	training's l2: 0.0804766	valid_1's l2: 0.769616
[391]	training's l2: 0.0801384	valid_1's l2: 0.768602
[392]	training's l2: 0.0797511

In [14]:
if run_experiment_II:
    y_pred_train = lgb.predict(X_train, num_iteration=lgb.best_iteration)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lgb.predict(X_val, num_iteration=lgb.best_iteration)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.09706488341239258
RMS log-error val:  0.2018445806471314
RMS log-error train (actual):  0.3953365923801808
RMS log-error val (actual):  0.8708139540731853


## Experiment III -- Random Forest (Too Slow)

In [15]:
run_experiment_III = True
if run_experiment_III:
    random_forest = RandomForestRegressor(random_state=1)
    random_forest.fit(X_train, y_train.values.ravel())

In [16]:
if run_experiment_III:
    y_pred_train = random_forest.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = random_forest.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.07991230493098243
RMS log-error val:  0.198402429075905
RMS log-error train (actual):  0.29627903576647857
RMS log-error val (actual):  0.8535536261784764


## Experiment IV -- XGBoost

In [17]:
run_experiment_IV = True
if run_experiment_IV:
    xgb = xgboost.XGBRegressor(n_estimators=20, early_stopping_rounds=100, learning_rate=0.3)
    xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True)

[0]	validation_0-rmse:4.31209	validation_1-rmse:4.31876
[1]	validation_0-rmse:3.06096	validation_1-rmse:3.08910
[2]	validation_0-rmse:2.19004	validation_1-rmse:2.24533
[3]	validation_0-rmse:1.59466	validation_1-rmse:1.68469
[4]	validation_0-rmse:1.18722	validation_1-rmse:1.32665
[5]	validation_0-rmse:0.91980	validation_1-rmse:1.11358
[6]	validation_0-rmse:0.73413	validation_1-rmse:0.98363
[7]	validation_0-rmse:0.61259	validation_1-rmse:0.92019
[8]	validation_0-rmse:0.53715	validation_1-rmse:0.88344
[9]	validation_0-rmse:0.48487	validation_1-rmse:0.86593
[10]	validation_0-rmse:0.45575	validation_1-rmse:0.85720
[11]	validation_0-rmse:0.42824	validation_1-rmse:0.85934
[12]	validation_0-rmse:0.40771	validation_1-rmse:0.85419
[13]	validation_0-rmse:0.39713	validation_1-rmse:0.85307
[14]	validation_0-rmse:0.37414	validation_1-rmse:0.85326
[15]	validation_0-rmse:0.36859	validation_1-rmse:0.85377
[16]	validation_0-rmse:0.34950	validation_1-rmse:0.85960
[17]	validation_0-rmse:0.34067	validation

In [18]:
if run_experiment_IV:
    y_pred_train = xgb.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = xgb.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.09657077521238491
RMS log-error val:  0.19491349654872678
RMS log-error train (actual):  0.3971298372479599
RMS log-error val (actual):  0.8530671459927017


In [19]:
if run_experiment_IV:
    xgb.save_model("xgb.model")

## Experiment V -- SVR

In [20]:
from sklearn.svm import SVR
run_experiment_V = False  # slow
if run_experiment_V:
    # svr = SVR(C=1.0, epsilon=0.2)
    svr = SVR(kernel='rbf')
    svr.fit(X_train, y_train.values.ravel())

In [21]:
if run_experiment_V:
    y_pred_train = svr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = svr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Test (Moment of Truth)

In [22]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [23]:
y_pred_test = main_predict(lgb, X_test)

[6.796066   8.188501   6.73770381 8.1299742 ]
[6.49096638 7.6615989  6.5503878  7.61513576]
[6.18127423 7.38670381 6.71012241 7.65339628]
[6.39348775 7.74445424 6.72689436 7.83433177]
[5.78110772 7.00616237 6.7413249  7.82282064]
[6.54792113 7.71567454 6.657167   7.70359622]
[6.38460818 7.55797529 6.61716318 7.60029266]
[6.678503   8.11966272 6.71568936 8.14399525]
[6.53379403 7.61798097 6.56229728 7.60897089]
[6.18442467 7.34323001 6.7024197  7.58465175]
[6.47405237 7.64956907 6.76925998 7.82708655]
[5.71600511 7.04749486 6.7413249  7.8259155 ]
[6.56600567 7.80389605 6.68898411 7.79301234]
[6.38942367 7.51850691 6.63813854 7.61855276]
[6.6569364  8.14914337 6.72041857 8.13660534]
[6.566807   7.60225586 6.64780762 7.64615229]


In [24]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': max(y_pred_test, 0)})
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submission.csv', index=False)